In [10]:
import quandl
import yfinance as yf
import cryptocompare
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
quandl.ApiConfig.api_key = 'YOUR_QUANDL_API_KEY'


For this report, I would like to predict The price of Arbitrium versus USD. Arbitrium is a relatively new L2 project constructed on the Ethereum Ecosystem. With the latest update, the transaction fees on Arbitrium One Chain is incredibly low (About 30 cents), therefore there is a big hype in the ecosystem. The main drivers that can effect the price of Arbitrium agains USD can be given as:
1. The Price of Bitcoin
2. The Price of Ethereum
3. Dollar Index (DXY)
4. S&P 500 Index (SPX)
5. Volatility Index (VIX)
6. Total Market CAP (Total)
7. Total Market CAP Excluding Bitcoin (Total2)
8. Total Market CAP Excluding Bitcoin and Ethereum (Total3)
9. Bitcoin Dominance (BTC.D)
10. USDT Dominance (USDT.D)

In [13]:
import yfinance as yf
import cryptocompare
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Fetch data from cryptocompare
bitcoin_data = cryptocompare.get_historical_price_day('BTC', currency='USD', limit=365)
ethereum_data = cryptocompare.get_historical_price_day('ETH', currency='USD', limit=365)

# Fetch S&P 500 (SPX) and VIX data using yfinance
spx_data = yf.download('^GSPC', start='2023-04-20', end='2024-04-20')[['Close']]
vix_data = yf.download('^VIX', start='2023-04-20', end='2024-04-20')[['Close']]

# Convert cryptocompare data to DataFrame
bitcoin_df = pd.DataFrame(bitcoin_data['Data']).set_index('time')
ethereum_df = pd.DataFrame(ethereum_data['Data']).set_index('time')

# Extract 'close' price
bitcoin_df = bitcoin_df['close']
ethereum_df = ethereum_df['close']

# Convert yfinance data to DataFrame
spx_df = spx_data['Close']
vix_df = vix_data['Close']

# Concatenate all the data into one DataFrame
data = pd.concat([bitcoin_df, ethereum_df, spx_df, vix_df], axis=1)
data.columns = ['BTC/USD', 'ETH/USD', 'SPX', 'VIX']

# Drop any missing values
data.dropna(inplace=True)

# Check the first few rows of the data
print(data.head())

# Split the data into training and test sets
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Fit the VAR model
model = VAR(train_scaled)
model_fitted = model.fit(maxlags=15)

# Forecast
forecast = model_fitted.forecast(train_scaled[-model_fitted.k_ar:], steps=len(test))

# Inverse transform the forecasted data
forecast_inv = scaler.inverse_transform(forecast)
forecast_df = pd.DataFrame(forecast_inv, index=test.index, columns=data.columns)

# Plot the actual vs forecasted values
plt.figure(figsize=(15, 6))
for column in data.columns:
    plt.plot(data.index, data[column], label=column)
    plt.plot(forecast_df.index, forecast_df[column], label=f"Forecast {column}", linestyle='--')
plt.title('Actual vs Forecasted Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


TypeError: list indices must be integers or slices, not str